In [1]:
from datetime import datetime

import numpy as np
import pandas as pd

from utils import Paths

# Load data

In [2]:
raw_lunch = pd.read_csv(Paths.raw_Sold_lunches(), delimiter=';', encoding='utf-8-sig', parse_dates=['Date'], low_memory=False)

raw_lunch.head()

,Date,Receipt time,Restaurant,Food Category,Dish,pcs,Hiilijalanjälki
0,2.1.2023,10:31,600 Chemicum,Liha,"Uunimakkaraa,sinappikastiketta",1,"0,9"
1,2.1.2023,10:32,600 Chemicum,Kala,Kalapuikot tillikermaviilikast,1,"1,04"
2,2.1.2023,10:32,600 Chemicum,Liha,"Uunimakkaraa,sinappikastiketta",1,"0,9"
3,2.1.2023,10:35,600 Chemicum,Kala,Kalapuikot tillikermaviilikast,1,"1,04"
4,2.1.2023,10:36,600 Chemicum,Liha,"Uunimakkaraa,sinappikastiketta",2,"1,8"


In [3]:
biowaste = pd.read_csv(Paths.raw_Biowaste(), delimiter=';')

biowaste.head()

,Date,Ravintola,Asiakasbiojäte. tiski (kg),Biojäte kahvi. porot (kg),Keittiön biojäte (ruoanvalmistus) (kg),Salin biojäte (jämät) (kg)
0,1.1.2023,600 Chemicum,0.0,0.0,0.0,0.0
1,1.1.2023,610 Physicum,0.0,0.0,0.0,0.0
2,1.1.2023,620 Exactum,0.0,0.0,0.0,0.0
3,2.1.2023,600 Chemicum,4.7,1.2,12.0,0.0
4,2.1.2023,610 Physicum,0.0,0.0,0.0,0.0


In [4]:
path = "../src/data/basic_mvp_data/tuntidata2.xlsx"

receipts = pd.read_excel(path, header=0)

receipts.head()

,Date,Kuitin tunti,Ravintola,Kuitti kpl
0,2023-01-02,8,600 Chemicum,1
1,2023-01-02,10,600 Chemicum,18
2,2023-01-02,11,600 Chemicum,83
3,2023-01-02,12,600 Chemicum,79
4,2023-01-02,13,600 Chemicum,90


In [5]:
occupancy = pd.read_excel(Paths.raw_occupancy(), header=0, index_col=None)

occupancy.head()

,dateCreated,countIn,countOut,phoneName
0,2024-03-15T04:26:15.000Z,0,1,S63
1,2024-03-15T04:26:24.000Z,1,2,S63
2,2024-03-15T04:26:27.000Z,1,0,S63
3,2024-03-15T05:19:19.000Z,1,0,S63
4,2024-03-15T05:37:09.000Z,1,0,S63


# 2. Process

## With `biowaste`

In [6]:
biowaste['Date'] = biowaste['Date'].apply(lambda x: datetime.strptime(x, r"%d.%m.%Y"))

biowaste.loc[:, 'Ravintola'] = biowaste['Ravintola'].replace({
    "600 Chemicum": "Chemicum",
    "610 Physicum": "Physicum",
    "620 Exactum": "Exactum"
})

biowaste = biowaste.rename(columns={
    'Date': 'date',
    'Ravintola': 'restaurant', 
    'Asiakasbiojäte. tiski (kg)': 'amnt_waste_customer', 
    'Biojäte kahvi. porot (kg)': 'amnt_waste_coffee', 
    'Keittiön biojäte (ruoanvalmistus) (kg)': 'amnt_waste_kitchen',
    'Salin biojäte (jämät) (kg)': 'amnt_waste_hall',
})

# Filter out rows which all 4 types of waste have zero amount
cols = ['amnt_waste_customer', 'amnt_waste_coffee', 'amnt_waste_kitchen', 'amnt_waste_hall']
tmp = biowaste[cols].sum(axis=1)
biowaste = biowaste[tmp > 0]

biowaste.head()

,date,restaurant,amnt_waste_customer,amnt_waste_coffee,amnt_waste_kitchen,amnt_waste_hall
3,2023-01-02,Chemicum,4.70,1.2,12.0,0.0
6,2023-01-03,Chemicum,5.00,1.4,14.8,0.0
9,2023-01-04,Chemicum,4.15,4.0,7.1,0.0
12,2023-01-05,Chemicum,10.00,3.3,8.5,0.0
24,2023-01-09,Chemicum,7.65,2.1,4.9,0.0


## With `receipts`

In [7]:
receipts.loc[:, 'Ravintola'] = receipts['Ravintola'].replace({
    "600 Chemicum": "Chemicum",
    "610 Physicum": "Physicum",
    "620 Exactum": "Exactum"
})
    
receipts = receipts.rename(columns={
    'Date': 'date',
    'Kuitin tunti': 'hour',
    'Ravintola': 'restaurant',
    'Kuitti kpl': 'num_rcpts'
})

receipts = receipts.groupby(['date', 'restaurant'])['num_rcpts'].sum().reset_index()

receipts.head()

,date,restaurant,num_rcpts
0,2023-01-02,Chemicum,272
1,2023-01-03,Chemicum,327
2,2023-01-04,Chemicum,351
3,2023-01-04,Physicum,1
4,2023-01-05,Chemicum,437


## With `occupancy`

In [8]:
phoneName2restaurant = {
    'S163': 'Exactum',
    'S216': 'Chemicum',
    'S217': 'Physicum',
}

occupancy = occupancy[occupancy['phoneName'].isin(phoneName2restaurant.keys())]
occupancy['restaurant'] = occupancy['phoneName'].apply(lambda x: phoneName2restaurant[x])

# Convert index
occupancy['date'] = pd.to_datetime(occupancy['dateCreated']).dt.tz_localize(None)
occupancy = occupancy.set_index('date')

# Accumulate countIn, countOut by day
cols = ['countIn', 'countOut', 'restaurant']
occupancy = occupancy[cols].groupby(by='restaurant').resample('D').sum()

# Reset date
occupancy = occupancy[['countIn', 'countOut']].reset_index()

# Rename columns' name
occupancy = occupancy.rename(columns={
    'countIn': 'num_customer_in',
    'countOut': 'num_customer_out'
})

occupancy.head()

,restaurant,date,num_customer_in,num_customer_out
0,Chemicum,2024-05-27,11,6
1,Chemicum,2024-05-28,767,709
2,Chemicum,2024-05-29,671,677
3,Chemicum,2024-05-30,717,697
4,Chemicum,2024-05-31,689,668


## With 'lunches'

In [10]:
lunches = (
    raw_lunch
    .replace(
        {
            '600 Chemicum': 'Chemicum',
            '610 Physicum': 'Physicum',
            '620 Exactum': 'Exactum',
            'Kala': 'fish',
            'Liha': 'meat',
            'Vegaani': 'vegan',
            'Kasvis': 'vegetarian',
            'Kana': 'chicken'
        },
    )
    .rename(columns={
        'Date': 'date',
        'Restaurant': 'restaurant',
        'Food Category': 'category',
        'Dish': 'dish'
    })
    
)

lunches['pcs'] = pd.to_numeric(lunches.pcs, errors='coerce')
lunches['date'] = pd.to_datetime(lunches.date, format='%d.%m.%Y', errors='coerce')

lunches = (
    lunches
    .groupby(['date', 'restaurant', 'category', 'dish'])['pcs']
    .sum()
    .reset_index()
)

# Remove 'Not Napped'
lunches = lunches[lunches['category'] != 'Not Mapped']

# Remove 'takeaway'
def is_takeaway(s: str):
    return s.lower().count('take away') > 0

lunches['is_takeaway'] = lunches['dish'].map(is_takeaway)
lunches = lunches[~lunches['is_takeaway']].drop(columns='is_takeaway')

# Add dish name processing
def _f(s: str):
    s = s.split(',')[0]
    s = s.split('&')[0]
    s = s.strip()

    return s

lunches['dish'] = lunches['dish'].apply(_f)

# Sum pieces again since on some dates, same dish appears twice
lunches = lunches.groupby(['date', 'restaurant', 'category', 'dish'])['pcs'].sum().reset_index()


lunches.head(10)

,date,restaurant,category,meal,pcs
2,2023-01-02,Chemicum,fish,Kalapuikot tillikermaviilikast,78.0
4,2023-01-02,Chemicum,meat,Uunimakkaraa,165.0
5,2023-01-02,Chemicum,vegan,Marokkolainen linssipata,84.0
8,2023-01-03,Chemicum,fish,Herkkulohipihvit,105.0
10,2023-01-03,Chemicum,fish,Kalapuikot tillikermaviilikast,52.0
12,2023-01-03,Chemicum,meat,Pasta Carbonara,17.0
13,2023-01-03,Chemicum,meat,Uunimakkaraa,56.0
14,2023-01-03,Chemicum,vegan,Marokkolainen linssipata,62.0
16,2023-01-03,Chemicum,vegan,Vegaaninen buttertofu,51.0
17,2023-01-03,Chemicum,vegetarian,Feta-pinaattilasagnette,29.0


# Create table 'fact'

## Create table containing dishes' name and meal quantity by date and restaurant

In [10]:
dish_names = (
    lunches
    .groupby(['date', 'restaurant', 'category'])
    .agg({
        'dish': lambda x: [x] if isinstance(x, str) else x,
    })
    .reset_index()
)

dish_names['category'] = dish_names['category'].map(lambda x: f"meals_{x}")
dish_names['dish'] = dish_names['dish'].map(lambda x: np.array([x]) if isinstance(x, str) else x)

dish_names = dish_names.pivot(index=['date', 'restaurant'], columns='category', values='dish')

dish_names.head()

,category,meals_chicken,meals_fish,meals_meat,meals_vegan,meals_vegetarian
date,restaurant,,,,,
2023-01-02,Chemicum,NaN,[Kalapuikot tillikermaviilikast],[Uunimakkaraa],[Marokkolainen linssipata],NaN
2023-01-03,Chemicum,NaN,"[Herkkulohipihvit, Kalapuikot tillikermaviilik...","[Pasta Carbonara, Uunimakkaraa]","[Marokkolainen linssipata, Vegaaninen buttertofu]",[Feta-pinaattilasagnette]
2023-01-04,Chemicum,NaN,"[Herkkulohipihvit, Rapea kalaleike]",[Lihapullat],[Punajuuripyörykät],NaN
2023-01-05,Chemicum,[Kievin kana],"[Rapea kalaleike, Sitruunaiset kalapalat]",NaN,[Meksikon Beanit Chilipata],NaN
2023-01-09,Chemicum,NaN,"[Kalapuikot tillikermaviilikast, Sitruunaiset ...",[Chorizo lihap ja ruskkastike],"[Kasvis-jalapnuget ja tomatsals, Kasvisjauhisp...",NaN


## Merge dims to create fact table

In [11]:
RESTAURANTS = ['Chemicum', 'Exactum', 'Physicum']
date_start = pd.to_datetime('2023-01-02')
date_end = pd.to_datetime('2024-07-02')

date_range = pd.date_range(date_start, date_end, freq='B')

fact = pd.concat(
    [
        pd.DataFrame({
            'date': date_range,
            'restaurant': restaurant
        })
        for restaurant in RESTAURANTS
    ]
)

foreign_keys = ['restaurant', 'date']

fact = (
    fact
    .merge(dish_names, on=foreign_keys, how='inner')
    .merge(occupancy, on=foreign_keys, how='left')
    .merge(receipts, on=foreign_keys, how='inner')
    .merge(biowaste, on=foreign_keys, how='inner')
    .merge(lunches, on=foreign_keys, how='inner')
)

fact.head()

,date,restaurant,meals_chicken,meals_fish,meals_meat,meals_vegan,meals_vegetarian,num_customer_in,num_customer_out,num_rcpts,amnt_waste_customer,amnt_waste_coffee,amnt_waste_kitchen,amnt_waste_hall,category,meal,pcs
0,2023-01-02,Chemicum,NaN,[Kalapuikot tillikermaviilikast],[Uunimakkaraa],[Marokkolainen linssipata],NaN,NaN,NaN,272,4.7,1.2,12.0,0.0,fish,Kalapuikot tillikermaviilikast,78.0
1,2023-01-02,Chemicum,NaN,[Kalapuikot tillikermaviilikast],[Uunimakkaraa],[Marokkolainen linssipata],NaN,NaN,NaN,272,4.7,1.2,12.0,0.0,meat,Uunimakkaraa,165.0
2,2023-01-02,Chemicum,NaN,[Kalapuikot tillikermaviilikast],[Uunimakkaraa],[Marokkolainen linssipata],NaN,NaN,NaN,272,4.7,1.2,12.0,0.0,vegan,Marokkolainen linssipata,84.0
3,2023-01-03,Chemicum,NaN,"[Herkkulohipihvit, Kalapuikot tillikermaviilik...","[Pasta Carbonara, Uunimakkaraa]","[Marokkolainen linssipata, Vegaaninen buttertofu]",[Feta-pinaattilasagnette],NaN,NaN,327,5.0,1.4,14.8,0.0,fish,Herkkulohipihvit,105.0
4,2023-01-03,Chemicum,NaN,"[Herkkulohipihvit, Kalapuikot tillikermaviilik...","[Pasta Carbonara, Uunimakkaraa]","[Marokkolainen linssipata, Vegaaninen buttertofu]",[Feta-pinaattilasagnette],NaN,NaN,327,5.0,1.4,14.8,0.0,fish,Kalapuikot tillikermaviilikast,52.0


# Save tables

In [12]:
# lunches.to_excel(Paths.dim_lucnhes(), index=False)
biowaste.to_excel(Paths.dim_biowaste(), index=False)

In [13]:
# fact.to_excel(Paths.fact(), index=False)